### LeNet-5 模型

LeNet-5 模型, 它是第一个成功应用数字识别问题的卷积神经网络, 总共有 7 层;

#### 第一层:卷积层

+ 输入就是原始图像的像素, LeNet-5 模型接受的输入层大小为 32×32*1 
+ 卷积层过滤器的尺寸 5×5,深度 6, 不使用全 0 填充, 步长 1(+1也是分多少步)
+ 这一层的输出尺寸为 32-5+1 = 28, 深度为 6
+ 这个卷积层共有 5×5×1×6+6 = 156个参数 其中 6 个为偏置项参数
+ 下一层节点矩阵为 28 ×28*6=4704 个节点, 每个节点和 5×5 =25 个当前层节点相连, 所以本层卷积层共有 4704×（25+1）=122304个连接点

#### 第二层:池化层

+ 这一层的输入是上层的输出, 28 ×28*6 的节点矩阵, 本层采取的过滤器大小 2×2 长和宽的步长 2 所以本层的输出矩阵的大小为 14×14×6

#### 第三层:卷积层

+ 本的输入矩阵 14×14×６使用的过滤器 5×5, 深度为16本层不使用全零填充,　步长为 1 本层的输出矩阵大小为 10×10×16　按照标准的卷积层本层应该有5×5×６×16=2416 个参数,　10×10×16×(25+1)=41600 个连接点, 

#### 第四层:池化层

+ 本层的输入矩阵大小为 10×10×16, 采用的过滤器为 2×2, 步长 ２,　本层的输入矩阵大小 5×５×16

#### 第五层:全连接层

+ 本层的输入矩阵大小为 5×5×16,　过滤器的大小 5×5　如果把 5×5×16 的节点拉成一个向量, 那么这一层就和全连接层的输入一样了, 本层的输出节点 120　总共 5×5×16×120+120=48120, 个参数

#### 第六层:全连接层

+ 本层的输入节点的个数为 120 个,　输出节点的个数为 84 总共的参数120×84+84=10164

#### 第七层:全连接层

+ 本层的输入节点 84 输出节点 10 总共 84×10+10=850 

### 程序实现 LeNet-5 模型

In [6]:
import tensorflow as tf

In [7]:
# 调整输入数据的格式, placeholder 的格式, 输入为一个四维矩阵

x = tf.placeholder(tf.float32,[
    BATCH_SIZE,mnist_inference.IMAGE_SIZE,
    mnist_inference.IMAGE_SIZE,
    mnist_inference.NUM_CHANNELS],NAME='x-input')

'''
+ 第一维表示一个 batch　中样例的个数
+ 第二维和第三维表示图片的尺寸
+ 第四维表示图片的深度, 对 RBG 格式的图片, 深度为 3
'''

## 类似的将输入的训练数据格式调整为一个四维矩阵,并将这个调整后的数据传入 sees.run 过程, 

reshaped_xs = np.reshape(xs,(BATCH_SIZE,
                            mnist_inference.IMAGE_SIZE,
                            mnist_inference.IMAGE_SIZE,
                            mnist_inference.NUM_CHANNELS))



NameError: name 'BATCH_SIZE' is not defined

In [ ]:
# 在调整完输入格式之后，只需要实现 LeNet-5 模型结构的前向传播过程

import tensorflow as tf

In [ ]:
## 配置神经网络的参数

INPUT_NODE = 784
OUTPUT_NODE = 10

IMAGE_SIZE = 28

NUM_CHANNELS = 1

NUM_LABELS = 10

## 第一层的卷积层的尺寸和深度

CONV1_DEEP = 32
CONV1_SIZE = 5

## 第二层的卷积层的尺寸和深度

CONV2_DEEP = 64
CONV2_SIZE = 5


## 全连接层的节点个数

FC_SIZE = 512


### 定义卷积神经网络的前向传播过程, 这里添加了一个新的参数,　train, 用于区分训练过程和测试过程,　在这个程序中将用到　dropout 
# 可以进一步提升模型可靠性并防止过拟合, dropout 过程只在训练时使用

def inference(input_tensor,train,regularizer):
    '''
    # 声明第一层卷积层的变量,　并实现前向传播过程,通过使用不同的命名空间来隔离不同层的变量,　这可以让每一层中的变量命名只需要
    考虑当前层的作用,　而不需要担心重命名的问题, 和标准的 LeNet-5 模型大不一样, 这里, 定义的卷积层输入 28 ×２８＊１的原始ＭNIST 
    图片像素, 因为卷积层中使用了全 0 填充, 所以, 所输出为 28×２８×32 的矩阵,
    '''
    with tf.variable_op_scope('layer1-conv1'):
        conv1_weights = tf.get_variable(
        'weight', [CONV1_SIZE,CONV1_SIZE,NUM_CHANNELS,CONV1_DEEP],
            initializer=tf.truncated_normal_initializer(stddev=0.1))
    conv1_biases = tf.get_variable('bias',[CONV1_DEEP],initializer=tf.constant_initializer(0.0))
    
    
    
    # 使用边长为 5 深度　32 的过滤器,　过滤器移动的步长为 1，　且使用全 0 填充,
    conv1 = tf.nn.conv2d(input_tensor,conv1_weights,strides=[1,1,1,1],padding='SAME')
    relu1 = tf.nn.relu(tf.nn.bias_add(conv1,conv1_biases))
    
    # 实现第二层的前向传播过程,　这里首先选用最大池化层,　池化层过滤器的边长为 2, 使用全0 填充且移动的步长为 2 这一层的输入是上一层的
    # 输出, 也就是说, 28 × 28 × 32的矩阵, 输出为 14 × 14 × 32 的矩阵.
    
    with tf.name_scope('layer2-pool1'):
        pooll = tf.nn.max_pool(relu1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
        
    # 声明第三层卷积层的变量并实现前向传播过程,这一层的输入为 14×14*32 的矩阵, 输出的为 14 ×14 ×64 的矩阵
    with tf.variable_op_scope('layer3-conv2'):
        conv2_weights = tf.get_variable('weight',[CONV2_SIZE,CONV2_SIZE,CONV1_DEEP,CONV2_DEEP],
                                        initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv2_biases = tf.get_variable('bias',[CONV2_DEEP],initializer=tf.constant_initializer(0.0))
        
        
    # 使用编程为 5 深度为 64 的过滤器, 过滤器移动的步长为 1 且全填0
    
        conv2 = tf.nn.conv2d(pooll,conv2_weights,strides=[1,1,1,1],padding='SAME')
        relu2 = tf.nn.relu(tf.nn.bias_add(conv2,conv2_biases))

        
    # 实现第 4 层池化层的前传播过程, 这一层和第二层的结构是一样的, 这一层的输入为 14 ×14× 64 的矩阵, 输出的矩阵为 7 × 7 ×64 的矩阵
    
    with tf.name_scope('layer4-pool2'):
        pool2 = tf.nn.max_pool(relu2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
        
        # 将第四层池化层的输出转化为第5 
        
        
    pool_shape = pool2.get_shape().as_list()
    
    nodes = pool_shape[1] * pool_shape[2] * pool_shape[3]
    
    reshaped = tf.reshape(pool2,[pool_shape[0],nodes])
    
    # 第五层
    
    with tf.variable_scope('layer5-fc1'):
        fc1_weights = tf.get_variable('weight',[nodes,FC_SIZE],initializer=tf.truncated_normal_initializer(stdded=0.1))
        # 只有全连接层的需要加入正则化
        
        if regularizer !=None:
            tf.add_to_collection('losses',regularizer(fc1_weights))
            
        fc1_biases = tf.get_variable('bias',[FC_SIZE],initializer=tf.constant_initializer(0.1))
            
        fc1 = tf.nn.relu(tf.matmul(reshaped,fc1_weights)+fc1_biases)
            
        if train:fc1=tf.nn.dropout(fc1,0.5)
            
    # 第六 层
    
    with tf.variable_scope('layer6-fc2'):
        fc2_weights = tf.get_variable('weight',[FC_SIZE,NUM_LABELS],initializer=tf.truncated_normal_initializer(stddev =0.1))
        if regularizer !=None:
            tf.add_to_collection('losses',regularizer(fc2_weights))
        
        fc2_biases = tf.get_variable('bias',[NUM_LABELS],initializer=tf.constant_initializer(0.1))
        logit = tf.matmul(fc1,fc2_weights) + fc2_biases
        
    return logit
